In [2]:
# %% Imports et initialisation

import os
import sys

import numpy as np
import torch
import torch.nn as nn
from PIL import Image

import gradio as gr

PROJECT_ROOT = os.path.abspath("..") 
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

from src.utils.config import load_config
from src.models.main_cnn import build_model
from src.data.datasets import EMOTION_LABELS 

print("PROJECT_ROOT =", PROJECT_ROOT)

# Paths to config and checkpoint
CONFIG_PATH = os.path.join(PROJECT_ROOT, "configs", "main_cnn_v1.yaml")
CHECKPOINT_PATH = os.path.join(PROJECT_ROOT, "experiments", "checkpoints", "best_model150epochs.pt")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device =", device)

# Chargement du config
cfg = load_config(CONFIG_PATH)

# Construction du modèle exactement comme en training
model = build_model(cfg.data).to(device)
model.eval()

# Chargement des poids (même format que dans main_train.py : "model_state")
print(f"Loading checkpoint from {CHECKPOINT_PATH}")
checkpoint = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=False)
state_dict = checkpoint["model_state"]
model.load_state_dict(state_dict)
print(f"Checkpoint chargé (epoch = {checkpoint.get('epoch', 'unknown')})")

PROJECT_ROOT = c:\Users\mathy\OneDrive - Groupe INSEEC (POCE)\Bureau\Centrale\Deep Learning\Deep_Learning_project
Device = cpu
Loading checkpoint from c:\Users\mathy\OneDrive - Groupe INSEEC (POCE)\Bureau\Centrale\Deep Learning\Deep_Learning_project\experiments\checkpoints\best_model150epochs.pt
Checkpoint chargé (epoch = 135)


In [3]:
# %% Préprocessing et prédiction

# Paramètres image issus du YAML
IMG_SIZE = int(cfg.data["dataset"].get("image_size", 128))
MEAN = np.array([0.5], dtype=np.float32)
STD = np.array([0.5], dtype=np.float32)

def preprocess_image(pil_img: Image.Image) -> torch.Tensor:
    """
    - Convertit en niveaux de gris
    - Resize à IMG_SIZE x IMG_SIZE
    - Normalise comme en training
    - Retourne un tenseur (1, 1, H, W) sur le device
    """
    # Conversion en niveaux de gris (1 canal)
    img = pil_img.convert("L")

    # Resize
    img = img.resize((IMG_SIZE, IMG_SIZE))

    # -> np.array [H, W], puis normalisation [0,1]
    arr = np.array(img).astype("float32") / 255.0  # (H, W)

    # Normalisation (x - mean)/std
    arr = (arr - MEAN[0]) / STD[0]

    # Ajout des dimensions channel + batch : (1, 1, H, W)
    tensor = torch.from_numpy(arr).unsqueeze(0).unsqueeze(0)

    # Envoi sur device
    tensor = tensor.to(device)
    return tensor


def predict_emotion(pil_img: Image.Image):
    """
    Fonction appelée par Gradio :
    renvoie l'étiquette d'émotion + confiance (%).
    """
    if pil_img is None:
        return "No image provided."

    x = preprocess_image(pil_img)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()

    top_idx = int(probs.argmax())
    top_label = EMOTION_LABELS[top_idx]
    top_conf = float(probs[top_idx])

    # Optionnel : construire un dict label -> prob pour debug
    # proba_dict = {label: float(p) for label, p in zip(EMOTION_LABELS, probs)}

    return f"Predicted emotion : {top_label} ({top_conf * 100:.2f} % confidence)"


In [ ]:
# %% Interface Gradio

iface = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Image(type="pil", label="Input Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="Emotion Recognition",
    description=(
        "Load an image of a face."
        "predicts the dominant emotion and the associated confidence."
    ),
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://73c2b8a7c22f25ed2b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
